### Imports

In [428]:
import numpy as np

### Install Cirq

In [429]:
try:
  import cirq
except ImportError:
  print("Installing Cirq...")
  !pip install --quiet cirq
  import cirq

  print("Cirq is now installed.")

### Helper Functions

In [430]:
def norm(vector):
  return np.linalg.norm(vector, 2)

def frobenius_norm(matrix):
  return np.linalg.norm(matrix, 'fro')

def ket(i, m):
  ket = []
  for j in range(m):
    if j == i:
      ket.append(1)
    else:
      ket.append(0)
  return ket

### $R_i$ Gate

In [431]:
class R(cirq.Gate):
  def __init__(self, i, t):
    super(R, self)
    self.i = i
    self.t = t

  def _num_qubits_(self):
    d = len(t)
    return ceil(log2(d))

  def _unitary_(self):
    t_i = [self.t[self.i]]
    I = np.eye(len(t_i))
    return I - (2 * np.outer(t_i, t_i))

  def _circuit_diagram_info_(self, args):
    return f"R{self.i}"

### Input

In [432]:
zero_state = [1, 0]
plus_state = [2**(-0.5), 2**(-0.5)]

A = [zero_state, plus_state]
print(f"A = {A}")

print()

m = len(zero_state)
d = len(A)

print(f"m = {m}")
print(f"d = {d}")

print()

A_F = frobenius_norm(A)
print(f"|A|_F = {A_F}")


A = [[1, 0], [0.7071067811865476, 0.7071067811865476]]

m = 2
d = 2

|A|_F = 1.4142135623730951


### $V_A$

In [433]:
V_A = []

# Resize it to the correct shape and fill with 0s.
for i in range(m):
  V_A.append(0)

for i in range(m):
  print(f"A_{i} = {A[i]}")

  A_i = norm(A[i])
  print(f"|A_{i}| = {A_i}")

  ket_i = ket(i, m)
  print(f"|{i}〉= {ket_i}")

  # The amplitude (or coefficient) of each ket is the ratio between |A_i| and |A|_F.
  # This is to renormalise each vector, ensuring unit length.
  amplitude = A_i / A_F
  print(f"|A_i|/|A|_F = {amplitude}")

  for j in range(m):
    ket_i[j] *= amplitude
  print(f"(|A_{i}|/|A|_F)|{i}〉= {ket_i}")

  print()

  for j in range(m):
    V_A[j] += ket_i[j]

print(f"V_A = {V_A}")

A_0 = [1, 0]
|A_0| = 1.0
|0〉= [1, 0]
|A_i|/|A|_F = 0.7071067811865475
(|A_0|/|A|_F)|0〉= [0.7071067811865475, 0.0]

A_1 = [0.7071067811865476, 0.7071067811865476]
|A_1| = 1.0
|1〉= [0, 1]
|A_i|/|A|_F = 0.7071067811865475
(|A_1|/|A|_F)|1〉= [0.0, 0.7071067811865475]

V_A = [0.7071067811865475, 0.7071067811865475]


### $U_A$

In [434]:
for i in range(m):
  U_A = []
  for q in range(m * d):
    U_A.append(0)
  print("U_A for i = ", i, " (initially): ", U_A)
  ket_i = []
  for x in range(m):
    if x == i:
      ket_i.append(1)
    else:
      ket_i.append(0)
  for j in range(d):
    print("A_", i, ": ", A[i])
    # print("Goooglooo")🤺
    print("A_", i, j, ": ", A[i][j])
    print("|", i, "〉: ", ket_i)
    ket_j = []
    for k in range(d):
      if k == j:
        ket_j.append(1)
      else:
        ket_j.append(0)
    print("|", j, "〉: ", ket_j)
    A_i = norm(A[i])
    print("|| A_", i, " ||: ", A_i)
    A_ij_divided_by_A_i = A[i][j]/A_i
    print("A_", i, j, " / || A_", i, " ||: ", A[i][j], "/", A_i, " = ", A_ij_divided_by_A_i)
    # Tensor ka function banana🍌 hoga
    ket_i_tensor_ket_j = []
    for l in range(m):
      for n in range(d):
        ket_i_tensor_ket_j.append(ket_i[l] * ket_j[n])
    print("|", i, "〉|", j, "〉: ", ket_i_tensor_ket_j)
    for p in range(m * d):
      ket_i_tensor_ket_j[p] = A_ij_divided_by_A_i * ket_i_tensor_ket_j[p]
    print("(A_", i, j, " / || A_", i, " ||)*|", i, "〉|", j, "〉: (", A[i][j], "/", A_i, ")*(|", i, "〉|", j, "〉) = ", ket_i_tensor_ket_j, "\n")
    for r in range(m * d):
      U_A[r] = U_A[r] + ket_i_tensor_ket_j[r]
  print("U_A for i = ", i, ": ", U_A, "\n")

U_A for i =  0  (initially):  [0, 0, 0, 0]
A_ 0 :  [1, 0]
A_ 0 0 :  1
| 0 〉:  [1, 0]
| 0 〉:  [1, 0]
|| A_ 0  ||:  1.0
A_ 0 0  / || A_ 0  ||:  1 / 1.0  =  1.0
| 0 〉| 0 〉:  [1, 0, 0, 0]
(A_ 0 0  / || A_ 0  ||)*| 0 〉| 0 〉: ( 1 / 1.0 )*(| 0 〉| 0 〉) =  [1.0, 0.0, 0.0, 0.0] 

A_ 0 :  [1, 0]
A_ 0 1 :  0
| 0 〉:  [1, 0]
| 1 〉:  [0, 1]
|| A_ 0  ||:  1.0
A_ 0 1  / || A_ 0  ||:  0 / 1.0  =  0.0
| 0 〉| 1 〉:  [0, 1, 0, 0]
(A_ 0 1  / || A_ 0  ||)*| 0 〉| 1 〉: ( 0 / 1.0 )*(| 0 〉| 1 〉) =  [0.0, 0.0, 0.0, 0.0] 

U_A for i =  0 :  [1.0, 0.0, 0.0, 0.0] 

U_A for i =  1  (initially):  [0, 0, 0, 0]
A_ 1 :  [0.7071067811865476, 0.7071067811865476]
A_ 1 0 :  0.7071067811865476
| 1 〉:  [0, 1]
| 0 〉:  [1, 0]
|| A_ 1  ||:  1.0
A_ 1 0  / || A_ 1  ||:  0.7071067811865476 / 1.0  =  0.7071067811865476
| 1 〉| 0 〉:  [0, 0, 1, 0]
(A_ 1 0  / || A_ 1  ||)*| 1 〉| 0 〉: ( 0.7071067811865476 / 1.0 )*(| 1 〉| 0 〉) =  [0.0, 0.0, 0.7071067811865476, 0.0] 

A_ 1 :  [0.7071067811865476, 0.7071067811865476]
A_ 1 1 :  0.7071067811865

In [435]:
# Determinants of n x n matrices using numpy
A = [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]]
determinant_A = np.linalg.det(A)
print(A)
print(determinant_A)

B = [[1, 0, 4, -6],
    [2, 5, 0, 3],
    [2, 0, 8, -12],
    [2, 1, -2, 3]]
determinant_B = np.linalg.det(B)
print(B)
print(determinant_B)

C = [[1, 1, -1],
     [1, 2, 2],
     [0, 3, 4]]
determinant_C = np.linalg.det(C)
print(C)
print(determinant_C)


X = [[0.5, 0.5, 0.5, 0.5],
     [0.5, -0.5, 0.5, 0.5],
     [0.5, 0.5, -0.5, 0.5],
     [0.5, -0.5, -0.5, -0.5]]
determinant_X = np.linalg.det(X)
print(X)
print(determinant_X)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
0.0
[[1, 0, 4, -6], [2, 5, 0, 3], [2, 0, 8, -12], [2, 1, -2, 3]]
0.0
[[1, 1, -1], [1, 2, 2], [0, 3, 4]]
-5.000000000000001
[[0.5, 0.5, 0.5, 0.5], [0.5, -0.5, 0.5, 0.5], [0.5, 0.5, -0.5, 0.5], [0.5, -0.5, -0.5, -0.5]]
-0.5


In [436]:
# Dot product of vectors
# print(len(X))

X = [[0.5, 0.5, 0.5, 0.5],
     [0.5, -0.5, 0.5, 0.5],
     [0.5, 0.5, -0.5, 0.5],
     [0.5, -0.5, -0.5, -0.5]]
print("X: ", X)

n = len(X)
transpose_X = [] # To access columns easily

for i in range(n):
  i_th_column = []
  for j in range(n):
    i_th_column.append(X[j][i])
  transpose_X.append(i_th_column)

print("Transpose of X: ", transpose_X, "(to access columns easily)")

for i in range(n):
  for j in range(n):
    if j > i and np.dot(transpose_X[i], transpose_X[j]) == 0:
      print("transpose_X[", i, "] = ", transpose_X[i], ", transpose_X[", j, "] = ", transpose_X[j]," are perpendicular")
    if j > i and np.dot(transpose_X[i], transpose_X[j]) != 0:
      print("transpose_X[", i, "] = ", transpose_X[i], ", transpose_X[", j, "] = ", transpose_X[j]," are not perpendicular")

X:  [[0.5, 0.5, 0.5, 0.5], [0.5, -0.5, 0.5, 0.5], [0.5, 0.5, -0.5, 0.5], [0.5, -0.5, -0.5, -0.5]]
Transpose of X:  [[0.5, 0.5, 0.5, 0.5], [0.5, -0.5, 0.5, -0.5], [0.5, 0.5, -0.5, -0.5], [0.5, 0.5, 0.5, -0.5]] (to access columns easily)
transpose_X[ 0 ] =  [0.5, 0.5, 0.5, 0.5] , transpose_X[ 1 ] =  [0.5, -0.5, 0.5, -0.5]  are perpendicular
transpose_X[ 0 ] =  [0.5, 0.5, 0.5, 0.5] , transpose_X[ 2 ] =  [0.5, 0.5, -0.5, -0.5]  are perpendicular
transpose_X[ 0 ] =  [0.5, 0.5, 0.5, 0.5] , transpose_X[ 3 ] =  [0.5, 0.5, 0.5, -0.5]  are not perpendicular
transpose_X[ 1 ] =  [0.5, -0.5, 0.5, -0.5] , transpose_X[ 2 ] =  [0.5, 0.5, -0.5, -0.5]  are perpendicular
transpose_X[ 1 ] =  [0.5, -0.5, 0.5, -0.5] , transpose_X[ 3 ] =  [0.5, 0.5, 0.5, -0.5]  are not perpendicular
transpose_X[ 2 ] =  [0.5, 0.5, -0.5, -0.5] , transpose_X[ 3 ] =  [0.5, 0.5, 0.5, -0.5]  are not perpendicular


In [437]:
# Projection Time!

def proj(u_j, v_i):
  inner_product_u_j_v_i_divided_by_inner_product_u_j_u_j = np.dot(v_i, u_j)/np.dot(u_j, u_j)
  # print("v, u", np.dot(v_i, u_j))
  # print("u, u", np.dot(u_j, u_j))
  # u_k = []
  # for x in range(u_j.size):
  #   # print(u_j[x])
  #   u_k.append(u_j[x])
  # for i in range(u_j.size):
  #   u_k[i] = u_k[i] * inner_product_u_j_v_i_divided_by_inner_product_u_j_u_j
  u_j = u_j * inner_product_u_j_v_i_divided_by_inner_product_u_j_u_j
  # print("u_k", u_k, "   u_j", u_j)
  # if (u_k == u_j).all():
  #   print("opa!")
  return u_j

# v4 - proj_u1(v4)- proj_u2(v4)- proj_u3(v4)
# proj_x(y) = ( ⟨y, x〉/ ⟨x, x〉)*x= ((y dot x)/(x dot x))*x

U = []

for i in range(n):
  # print(transpose_X[i])
  v_i = np.array(transpose_X[i])
  u_i = v_i
  print("v_", i, ": ", v_i)
  print("u_", i, ": ", u_i, "(initially)")
  # print(u_i)
  for j in range(n):
    if j < i:
      # print(transpose_X[i])
      # print(i, " ", j)
      print("proj_u_", j, "(v_", i, "): ", proj(U[j], v_i))
      u_i = u_i - proj(U[j], v_i)
  print("u_", i, ": ", u_i, "(final)")
  U.append(u_i)
  print("U : ", U, "\n")

# column1 = np.array([0.5, 0.5, 0.5, 0.5])
# print(np.dot(column1, column1))
print("U : ", U, "(Final)\n")

v_ 0 :  [0.5 0.5 0.5 0.5]
u_ 0 :  [0.5 0.5 0.5 0.5] (initially)
u_ 0 :  [0.5 0.5 0.5 0.5] (final)
U :  [array([0.5, 0.5, 0.5, 0.5])] 

v_ 1 :  [ 0.5 -0.5  0.5 -0.5]
u_ 1 :  [ 0.5 -0.5  0.5 -0.5] (initially)
proj_u_ 0 (v_ 1 ):  [0. 0. 0. 0.]
u_ 1 :  [ 0.5 -0.5  0.5 -0.5] (final)
U :  [array([0.5, 0.5, 0.5, 0.5]), array([ 0.5, -0.5,  0.5, -0.5])] 

v_ 2 :  [ 0.5  0.5 -0.5 -0.5]
u_ 2 :  [ 0.5  0.5 -0.5 -0.5] (initially)
proj_u_ 0 (v_ 2 ):  [0. 0. 0. 0.]
proj_u_ 1 (v_ 2 ):  [ 0. -0.  0. -0.]
u_ 2 :  [ 0.5  0.5 -0.5 -0.5] (final)
U :  [array([0.5, 0.5, 0.5, 0.5]), array([ 0.5, -0.5,  0.5, -0.5]), array([ 0.5,  0.5, -0.5, -0.5])] 

v_ 3 :  [ 0.5  0.5  0.5 -0.5]
u_ 3 :  [ 0.5  0.5  0.5 -0.5] (initially)
proj_u_ 0 (v_ 3 ):  [0.25 0.25 0.25 0.25]
proj_u_ 1 (v_ 3 ):  [ 0.25 -0.25  0.25 -0.25]
proj_u_ 2 (v_ 3 ):  [ 0.25  0.25 -0.25 -0.25]
u_ 3 :  [-0.25  0.25  0.25 -0.25] (final)
U :  [array([0.5, 0.5, 0.5, 0.5]), array([ 0.5, -0.5,  0.5, -0.5]), array([ 0.5,  0.5, -0.5, -0.5]), array([-0.25,  0.

In [438]:
# Doing Quantum Gram-Schmidt now

In [439]:
# 1. Define n
# 2. 2^n x (2^n + n)
# 3. First n columns populate truth-table-style
# 4. Make s's (might be able to reuse x's)
# 5. Input: size n all zeros ket
# 6. Apply Hadamard
# 7. Find Uf for all s's || x1.s1 XOR x2.s2...
# 8. Populate other columns
# 9. True/False orthogonal matrix

In [440]:
# n =  2 # No. of bits
# M = (1 << n) 	# No. of rows (2^n)
# N = (1 << n) + n 	# No. of columns (2^n + n)

# Table = [] # Final table

# # Making my final table
# for row in range(M):
#   Table.append([])
#   for column in range(N):
#     Table[row].append(0)

# print(Table)

In [441]:
# x = 3

# for col in range(x):
#   bit = '0'
#   count = 0
#   powerOfTwo = (1 << (x - 1 - col))
#   print('2^', col, ': ', powerOfTwo)
#   for i in range(1 << x):
#     if count >= powerOfTwo:
#       count = 0
#       if bit == '0':
#         bit = '1'
#       elif bit == '1':
#         bit = '0'
#     print(bit, ' count: ', count)
#     count = count + 1
#   print('\n')

In [442]:
# # Populating truth-table-style
# for column in range(n):
#   bit = '0'
#   count = 0
#   powerOfTwo = (1 << (n - 1 - column))
#   for row in range(M):
#     if count >= powerOfTwo:
#       count = 0
#       if bit == '0':
#         bit = '1'
#       elif bit == '1':
#         bit = '0'
#     # print(bit, ' count: ', count)
#     Table[row][column] = bit
#     count = count + 1

# for row in range(M):
#   print(Table[row], "\n")

In [443]:
# Populating truth-table-style
# for column in range(n):
#   for row in range(M):
#     Table[row][column] = str(row) + '' + str(column)



# Abandon
# print(Table)


# def generate_truth_table(n):
#     truth_table = []
#     max_value = M

#     for i in range(max_value):
#         binary_string = bin(i)[2:].zfill(n)  # Convert to binary string of length n
#         print(binary_string)
#         truth_table.append(list(map(int, list(binary_string))))

#     return truth_table

# # Example usage
# n = 3  # Specify the value of n
# result = generate_truth_table(n)
# print(result)


In [444]:
# for row in range(M):
#   X = []
#   for column in range(N):
#     if column < n:
#       # Define X
#       X.append(Table[row][column])
#     else # if column > (n - 1):
#       for x in range(M):
#         S = [] # Define S
#         for i in range(n):


In [445]:
# # Trying new approach keeping X tables

# JustXTable = []
# for row in range(M):
#   X = []
#   for column in range(n):
#     X.append(Table[row][column])
#   JustXTable.append(X)

# print("JustXTable: ", JustXTable, "\n")

# for X in range(M):
#   for S in range(M):
#     print("X: ", JustXTable[X])
#     print("S: ", JustXTable[S])
#     ListOfAnds = []
#     for i in range(n):
#       ListOfAnds.append(JustXTable[X][i] + '.' + JustXTable[S][i])
#     print("ListOfAnds: ", ListOfAnds)
#     XOROfListOfAnds = ''
#     for i in range(n):
#       if i == (n - 1):
#         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i]
#       else:
#         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i] + ' ⊕ '
#     print("XOROfListOfAnds: ", XOROfListOfAnds)
#     Table[X][S + n] = XOROfListOfAnds
#     print("\n")
#   print("\n\n")

# # print(Table)
# for row in range(M):
#   print(Table[row], "\n")
# #   for column in range(N):
# #     if column > (n - 1):



In [446]:
# n =  2 # No. of bits
# M = (1 << n) 	# No. of rows (2^n)
# N = (1 << n) + n 	# No. of columns (2^n + n)

# Table = [] # Final table

# for row in range(M):
#   Table.append([])
#   for column in range(N):
#     Table[row].append(0)

# for column in range(n):
#   bit = 0
#   count = 0
#   powerOfTwo = (1 << (n - 1 - column))
#   for row in range(M):
#     if count >= powerOfTwo:
#       count = 0
#       if bit == 0:
#         bit = 1
#       elif bit == 1:
#         bit = 0
#     Table[row][column] = bit
#     count = count + 1

# JustXTable = []
# for row in range(M):
#   X = []
#   for column in range(n):
#     X.append(Table[row][column])
#   JustXTable.append(X)

# for X in range(M):
#   for S in range(M):
#     ListOfAnds = []
#     for i in range(n):
#       ListOfAnds.append(str(JustXTable[X][i]) + '.' + str(JustXTable[S][i]))
#     XOROfListOfAnds = ''
#     for i in range(n):
#       if i == (n - 1):
#         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i]
#       else:
#         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i] + ' ⊕ '
#     Table[X][S + n] = XOROfListOfAnds

# print("Formula: \n")

# for row in range(M):
#   print(Table[row], "\n")


# # print("After calculating values: \n")

# # for row in range(M):
# #   print(Table[row], "\n")


In [447]:
# print(0 & 0)
# print(0 & 1)
# print(1 & 0)
# print(1 & 1)

In [448]:
# print(0 ^ 0)
# print(0 ^ 1)
# print(1 ^ 0)
# print(1 ^ 1)

In [449]:
# for X in range(M):
#   for S in range(M):
#     ListOfAnds = []
#     for i in range(n):
#       ListOfAnds.append(JustXTable[X][i] & JustXTable[S][i]) # ANDing xi and si
#     XOROfListOfAnds = ListOfAnds[0]
#     for i in range(n):
#       if i != 0:
#         XOROfListOfAnds = XOROfListOfAnds ^ ListOfAnds[i] # XORing xi.si with next xi.si
#     Table[X][S + n] = XOROfListOfAnds

# # for X in range(M):
# #   for S in range(M):
# #     ListOfAnds = []
# #     for i in range(n):
# #       ListOfAnds.append(JustXTable[X][i] + '.' + JustXTable[S][i])
# #     XOROfListOfAnds = ''
# #     for i in range(n):
# #       if i == (n - 1):
# #         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i]
# #       else:
# #         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i] + ' ⊕ '
# #     Table[X][S + n] = XOROfListOfAnds

# print("After calculating values: \n")

# for row in range(M):
#   print(Table[row], "\n")

In [450]:
# n =  2 # No. of bits
# M = (1 << n) 	# No. of rows (2^n)
# N = (1 << n) + n 	# No. of columns (2^n + n)

# Table = [] # Final table

# for row in range(M):
#   Table.append([])
#   for column in range(N):
#     Table[row].append(0)

# for column in range(n):
#   bit = 0
#   count = 0
#   powerOfTwo = (1 << (n - 1 - column))
#   for row in range(M):
#     if count >= powerOfTwo:
#       count = 0
#       if bit == 0:
#         bit = 1
#       elif bit == 1:
#         bit = 0
#     Table[row][column] = bit
#     count = count + 1

# JustXTable = []
# for row in range(M):
#   X = []
#   for column in range(n):
#     X.append(Table[row][column])
#   JustXTable.append(X)

# for X in range(M):
#   for S in range(M):
#     ListOfAnds = []
#     for i in range(n):
#       ListOfAnds.append(str(JustXTable[X][i]) + '.' + str(JustXTable[S][i]))
#     XOROfListOfAnds = ''
#     for i in range(n):
#       if i == (n - 1):
#         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i]
#       else:
#         XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i] + ' ⊕ '
#     Table[X][S + n] = XOROfListOfAnds

# print("Formula: \n")

# for row in range(M):
#   print(Table[row], "\n")

# print("After calculating values classically: \n")

# for X in range(M):
#   for S in range(M):
#     ListOfAnds = []
#     for i in range(n):
#       ListOfAnds.append(JustXTable[X][i] & JustXTable[S][i]) # ANDing xi and si
#     XOROfListOfAnds = ListOfAnds[0]
#     for i in range(n):
#       if i != 0:
#         XOROfListOfAnds = XOROfListOfAnds ^ ListOfAnds[i] # XORing xi.si with next xi.si
#     Table[X][S + n] = XOROfListOfAnds

# for row in range(M):
#   print(Table[row], "\n")

# print("After calculating values quantum-way: \n")

# for row in range(M):
#   for column in range(N):
#     if column > (n - 1):
#       if Table[row][column] == 0:
#         Table[row][column] = 0.5
#       elif Table[row][column] == 1:
#         Table[row][column] = -0.5

# for row in range(M):
#   print(Table[row], "\n")

# print("Determing if quantum-way matrix is orthogonal: \n")

# import numpy as np

# Matrix = []

# for row in range(M):
#   Row = []
#   for column in range(N):
#     if column > (n - 1):
#       Row.append(Table[row][column])
#   Matrix.append(Row)

# Matrix = np.array(Matrix)
# TransposeOfMatrix = Matrix.transpose()
# IdentityMatrix = np.identity(len(Matrix))

# if (np.allclose(np.dot(Matrix, TransposeOfMatrix), IdentityMatrix)) == True:
#   print('Matrix is orthogonal')
# else:
#   print('Matrix is not orthogonal')

# # print(Matrix)
# # print(TransposeOfMatrix)
# # for row in range(M):
# #   print(Matrix[row], "\n")



In [451]:
n =  3 # No. of bits
M = (1 << n) 	# No. of rows (2^n)
N = (1 << n) + n 	# No. of columns (2^n + n)

Table = [] # Final table

for row in range(M):
  Table.append([])
  for column in range(N):
    Table[row].append(0)

for column in range(n):
  bit = 0
  count = 0
  powerOfTwo = (1 << (n - 1 - column))
  for row in range(M):
    if count >= powerOfTwo:
      count = 0
      if bit == 0:
        bit = 1
      elif bit == 1:
        bit = 0
    Table[row][column] = bit
    count = count + 1

JustXTable = []
for row in range(M):
  X = []
  for column in range(n):
    X.append(Table[row][column])
  JustXTable.append(X)

for X in range(M):
  for S in range(M):
    ListOfAnds = []
    for i in range(n):
      ListOfAnds.append(str(JustXTable[X][i]) + '.' + str(JustXTable[S][i]))
    XOROfListOfAnds = ''
    for i in range(n):
      if i == (n - 1):
        XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i]
      else:
        XOROfListOfAnds = XOROfListOfAnds + ListOfAnds[i] + ' ⊕ '
    Table[X][S + n] = XOROfListOfAnds

print("Formula: \n")

for row in range(M):
  print(Table[row], "\n")

print("After calculating values classically: \n")

for X in range(M):
  for S in range(M):
    ListOfAnds = []
    for i in range(n):
      ListOfAnds.append(JustXTable[X][i] & JustXTable[S][i]) # ANDing xi and si
    XOROfListOfAnds = ListOfAnds[0]
    for i in range(n):
      if i != 0:
        XOROfListOfAnds = XOROfListOfAnds ^ ListOfAnds[i] # XORing xi.si with next xi.si
    Table[X][S + n] = XOROfListOfAnds

for row in range(M):
  print(Table[row], "\n")

print("After calculating values quantum-way: \n")

for row in range(M):
  for column in range(N):
    if column > (n - 1):
      if Table[row][column] == 0:
        Table[row][column] = 0.5
      elif Table[row][column] == 1:
        Table[row][column] = -0.5

for row in range(M):
  print(Table[row], "\n")

print("Determing if quantum-way matrix is orthogonal: \n")

import numpy as np

Matrix = []

for row in range(M):
  Row = []
  for column in range(N):
    if column > (n - 1):
      Row.append(Table[row][column])
  Matrix.append(Row)

Matrix = np.array(Matrix)
TransposeOfMatrix = Matrix.transpose()
IdentityMatrix = np.identity(len(Matrix))

print(Matrix, '\n', TransposeOfMatrix, '\n', IdentityMatrix, '\n')

verdict = True
for i in range(M):
  for j in range(M):
    if (abs(np.dot(TransposeOfMatrix[i], TransposeOfMatrix[j])) > 1e-9):
      verdict = False
      break
print(verdict)

# if (np.allclose(np.dot(Matrix, TransposeOfMatrix), IdentityMatrix)) == True:
#   print('Matrix is orthogonal')
# else:
#   print('Matrix is not orthogonal')

# if (abs(np.dot(Matrix, TransposeOfMatrix)) < 1e-9):
#   print('Matrix is orthogonal')
# else:
#   print('Matrix is not orthogonal')

Formula: 

[0, 0, 0, '0.0 ⊕ 0.0 ⊕ 0.0', '0.0 ⊕ 0.0 ⊕ 0.1', '0.0 ⊕ 0.1 ⊕ 0.0', '0.0 ⊕ 0.1 ⊕ 0.1', '0.1 ⊕ 0.0 ⊕ 0.0', '0.1 ⊕ 0.0 ⊕ 0.1', '0.1 ⊕ 0.1 ⊕ 0.0', '0.1 ⊕ 0.1 ⊕ 0.1'] 

[0, 0, 1, '0.0 ⊕ 0.0 ⊕ 1.0', '0.0 ⊕ 0.0 ⊕ 1.1', '0.0 ⊕ 0.1 ⊕ 1.0', '0.0 ⊕ 0.1 ⊕ 1.1', '0.1 ⊕ 0.0 ⊕ 1.0', '0.1 ⊕ 0.0 ⊕ 1.1', '0.1 ⊕ 0.1 ⊕ 1.0', '0.1 ⊕ 0.1 ⊕ 1.1'] 

[0, 1, 0, '0.0 ⊕ 1.0 ⊕ 0.0', '0.0 ⊕ 1.0 ⊕ 0.1', '0.0 ⊕ 1.1 ⊕ 0.0', '0.0 ⊕ 1.1 ⊕ 0.1', '0.1 ⊕ 1.0 ⊕ 0.0', '0.1 ⊕ 1.0 ⊕ 0.1', '0.1 ⊕ 1.1 ⊕ 0.0', '0.1 ⊕ 1.1 ⊕ 0.1'] 

[0, 1, 1, '0.0 ⊕ 1.0 ⊕ 1.0', '0.0 ⊕ 1.0 ⊕ 1.1', '0.0 ⊕ 1.1 ⊕ 1.0', '0.0 ⊕ 1.1 ⊕ 1.1', '0.1 ⊕ 1.0 ⊕ 1.0', '0.1 ⊕ 1.0 ⊕ 1.1', '0.1 ⊕ 1.1 ⊕ 1.0', '0.1 ⊕ 1.1 ⊕ 1.1'] 

[1, 0, 0, '1.0 ⊕ 0.0 ⊕ 0.0', '1.0 ⊕ 0.0 ⊕ 0.1', '1.0 ⊕ 0.1 ⊕ 0.0', '1.0 ⊕ 0.1 ⊕ 0.1', '1.1 ⊕ 0.0 ⊕ 0.0', '1.1 ⊕ 0.0 ⊕ 0.1', '1.1 ⊕ 0.1 ⊕ 0.0', '1.1 ⊕ 0.1 ⊕ 0.1'] 

[1, 0, 1, '1.0 ⊕ 0.0 ⊕ 1.0', '1.0 ⊕ 0.0 ⊕ 1.1', '1.0 ⊕ 0.1 ⊕ 1.0', '1.0 ⊕ 0.1 ⊕ 1.1', '1.1 ⊕ 0.0 ⊕ 1.0', '1.1 ⊕ 0.0 ⊕ 1.1', '1.1 ⊕ 0.1 ⊕ 1.0', '1.1 ⊕ 0.1 ⊕ 1.1'] 

[1, 1

In [475]:
# QGS with AND approach

n =  2 # No. of bits
M = (1 << n) 	# No. of rows (2^n)
N = (1 << n) + n 	# No. of columns (2^n + n)

Table = [] # Final table

for row in range(M):
  Table.append([])
  for column in range(N):
    Table[row].append(0)

for column in range(n):
  bit = 0
  count = 0
  powerOfTwo = (1 << (n - 1 - column))
  for row in range(M):
    if count >= powerOfTwo:
      count = 0
      if bit == 0:
        bit = 1
      elif bit == 1:
        bit = 0
    Table[row][column] = bit
    count = count + 1

JustXTable = []
for row in range(M):
  X = []
  for column in range(n):
    X.append(Table[row][column])
  JustXTable.append(X)

for X in range(M):
  for S in range(M):
    ListOfAnds = []
    for i in range(n):
      ListOfAnds.append(str(JustXTable[X][i]) + '.' + str(JustXTable[S][i]))
    AndOfListOfAnds = ''
    for i in range(n):
      if i == (n - 1):
        ANDOfListOfAnds = ANDOfListOfAnds + ListOfAnds[i]
      else:
        ANDOfListOfAnds = ANDOfListOfAnds + ListOfAnds[i] + ' ⊕ '
    Table[X][S + n] = ANDOfListOfAnds

print("Formula: \n")

for row in range(M):
  print(Table[row], "\n")

print("After calculating values classically: \n")

for X in range(M):
  for S in range(M):
    ListOfAnds = []
    for i in range(n):
      if JustXTable[S][i] == 1:
        ListOfAnds.append(JustXTable[X][i] & JustXTable[S][i]) # ANDing xi and si
      else:
        ListOfAnds.append(1)
    if len(ListOfAnds) != 0:
      ANDOfListOfAnds = ListOfAnds[0]
      for i in range(len(ListOfAnds)):
        if i != 0:
          ANDOfListOfAnds = ANDOfListOfAnds & ListOfAnds[i] # ANDing xi.si with next xi.si
      Table[X][S + n] = ANDOfListOfAnds

for row in range(M):
  print(Table[row], "\n")

print("After calculating values quantum-way: \n")

for row in range(M):
  for column in range(N):
    if column > (n - 1):
      if Table[row][column] == 0:
        Table[row][column] = 0.5
      elif Table[row][column] == 1:
        Table[row][column] = -0.5

for row in range(M):
  print(Table[row], "\n")


Formula: 

[0, 0, '0.0 ⊕ 0.0', '0.0 ⊕ 0.1', '0.1 ⊕ 0.0', '0.1 ⊕ 0.1'] 

[0, 1, '0.0 ⊕ 1.0', '0.0 ⊕ 1.1', '0.1 ⊕ 1.0', '0.1 ⊕ 1.1'] 

[1, 0, '1.0 ⊕ 0.0', '1.0 ⊕ 0.1', '1.1 ⊕ 0.0', '1.1 ⊕ 0.1'] 

[1, 1, '1.0 ⊕ 1.0', '1.0 ⊕ 1.1', '1.1 ⊕ 1.0', '1.1 ⊕ 1.1'] 

After calculating values classically: 

[0, 0, 1, 0, 0, 0] 

[0, 1, 1, 1, 0, 0] 

[1, 0, 1, 0, 1, 0] 

[1, 1, 1, 1, 1, 1] 

After calculating values quantum-way: 

[0, 0, -0.5, 0.5, 0.5, 0.5] 

[0, 1, -0.5, -0.5, 0.5, 0.5] 

[1, 0, -0.5, 0.5, -0.5, 0.5] 

[1, 1, -0.5, -0.5, -0.5, -0.5] 



In [452]:
# import numpy as np

# def f(s, x):
# 	product = 1
# 	for j in x:
# 		product *= j

# 	for j in s:
# 		product *= j

# 	return product

# def g(s, x):
# 	return np.dot(s, x) % 2

# def sweaty(s, x):
# 	# n choose 2, all pairs list
# 	pairs = [(i,j) for i in range(len(x)) for j in range(len(x)) if i < j]

# 	sum = 0
# 	s_index = 0

# 	for pair in pairs:
# 		sum += (x[pair[0]] * x[pair[1]] * s[s_index]) % 2
# 		s_index += 1

# 	for xi in x:
# 		sum += (s[s_index] * xi) % 2
# 		s_index += 1

# 	return sum

# def choose(n, r):
# 	"""
# 	Computes the binomial coefficient n choose r.
# 	"""
# 	return np.math.factorial(n) / (np.math.factorial(r) * np.math.factorial(n - r))


# xbits = 3
# sbits = 3
# matrix = np.zeros((2**xbits, 2**sbits))

# # Try all possible values of s and x for function f and store in matrix
# # Columns are secret strings.
# # Rows are inputs.
# for x in range(2**xbits):
# 	for s in range(2**sbits):
# 		s_list = [int(i) for i in bin(s)[2:].zfill(sbits)]
# 		x_list = [int(i) for i in bin(x)[2:].zfill(xbits)]
# 		value = g(s_list, x_list)

# 		matrix[x][s] = -0.5 if (value == 1) else 0.5


# print(matrix)

# def pairwise_orthogoanal(matrix):
# 	"""
# 	Checks whether each column of the matrix is orthogonal to each other.
# 	"""

# 	for i in range(len(matrix)):
# 		for j in range(len(matrix)):
# 			if (i != j):
# 				if (np.dot(matrix[:,i], matrix[:,j]) != 0):
# 					return False

# 	return True

# print(pairwise_orthogoanal(matrix))